In [20]:
import pandas as pd
import requests
from westgate.flaml_model import *

In [26]:
payload_example = {
  "gender": "male",
  "dob": "2000-03-24",
  "request_date": "2024-03-24",
  "account_age_days": 360,
  "count_nsf_90_days": 10,
  "count_nsf_30_days": 0,
  "count_stop_payment_90_days": 0,
  "sum_micro_loan_payments_current_month": 0,
  "sum_micro_loan_payments_previous_month": 0,
  "sum_micro_loan_payments_2_months_ago": 0,
  "recurring_deposits_current_month": 0,
  "recurring_deposits_previous_month": 0,
  "recurring_deposits_2_months_ago": 0,
  "count_active_days": 10,
  "sum_total_income_current_month": 1000,
  "sum_total_income_previous_month": 0,
  "sum_total_income_2_months_ago": 0,
  "sum_loan_payments_current_month": 0,
  "sum_loan_payments_previous_month": 0,
  "sum_loan_payments_2_months_ago": 0,
  "sum_loan_deposits_30_days": 0,
  "sum_loan_deposits_60_days": 0,
  "sum_loan_deposits_90_days": 10,
  "sum_government_income_current_month": 0,
  "sum_government_income_previous_month": 0,
  "sum_government_income_2_months_ago": 0,
  "sum_employment_insurance_income_current_month": 0,
  "sum_employment_insurance_income_previous_month": 0,
  "sum_employment_insurance_income_2_months_ago": 0,
  "sum_employer_income_current_month": 0,
  "sum_employer_income_previous_month": 0,
  "sum_employer_income_2_months_ago": 0,
  "sum_disability_income_current_month": 0,
  "sum_disability_income_previous_month": 0,
  "sum_disability_income_2_months_ago": 0,
  "average_monthly_recurring_transfers_in_complex": 0,
  "average_monthly_recurring_transfers_out_complex": 0,
  "employer_income_frequency": "BiWeekly",
  "average_closing_balance_day_after_employer_income": 1250,
  "average_closing_balance_day_of_employer_income": 1000,
  "balance_90_days_ago": 90,
  "balance_current": 1000,
  "balance_min": 0,
  "count_loan_payments_current_month": 0,
  "count_loan_payments_previous_month": 3,
  "count_distinct_micro_lenders": 0,
  "count_days_negative_balance_90_days": 0,
  "sum_non_employer_income_current_month": 0,
  "sum_non_employer_income_previous_month": 0,
  "sum_non_employer_income_2_months_ago": 0,
  "sum_non_employer_income": 0,
  "average_closing_balance_day_of_income": 0,
  "average_monthly_micro_loan_payments_complex": 0,
  "average_monthly_utility_payments_complex": 0,
  "telecom_payments_average": 0,
  "other_loan_payments_average": 0,
  "utility_payments_average": 0,
  "zozo": 0
}

In [15]:
accepted_df = pd.read_csv('../data/AttributesLoans2022.csv', encoding='latin')
refused_raw_df = pd.read_csv('../data/refusal2022.csv', encoding='latin')

C:\Users\karim\AppData\Local\Temp\ipykernel_10268\3967161521.py:1: DtypeWarning: Columns (741) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv('../data/AttributesLoans2022.csv', encoding='latin')


In [16]:
refused_raw_df['recurring_deposits_90_days'] = (
    refused_raw_df['recurring_deposits_current_month'] + 
    refused_raw_df['recurring_deposits_previous_month'] +
    refused_raw_df['recurring_deposits_2_months_ago']
)

refused_raw_df['sum_micro_loans_60_days'] = (
    refused_raw_df['sum_micro_loan_payments_current_month'] + 
    refused_raw_df['sum_micro_loan_payments_previous_month'] 
)

refused_raw_df['recurring_deposits_90_days'] = (
    refused_raw_df['recurring_deposits_current_month'] + 
    refused_raw_df['recurring_deposits_previous_month'] +
    refused_raw_df['recurring_deposits_2_months_ago']
)

refused_raw_df['sum_micro_loans_60_days'] = (
    refused_raw_df['sum_micro_loan_payments_current_month'] + 
    refused_raw_df['sum_micro_loan_payments_previous_month'] 
)

In [17]:
print('# loans before auto-refusal: ' + str(len(refused_raw_df)))

refused_df = refused_raw_df[refused_raw_df['account_age_days'] >= 85]

refused_df = refused_df[refused_df['count_nsf_90_days'] <= 8]

refused_df = refused_df[refused_df['count_nsf_30_days'] <= 6]

refused_df = refused_df[refused_df['count_stop_payment_90_days'] <= 4]

#refused_df = refused_df[refused_df['recurring_deposits_90_days']/3.0 >= 1800] <-- filers way too much

#refused_df = refused_df[refused_df['sum_micro_loans_60_days'] <= 1000]

print('# loans after auto-refusal: ' + str(len(refused_df)))

# loans before auto-refusal: 37398
# loans after auto-refusal: 28579


In [22]:
default_model = load_model(experiment_id='default_1.0', basefolder='../default_model/')

In [65]:
refusal_model = load_model(experiment_id='refusal_0.2', basefolder='../refusal_model/')

In [71]:
cols_default = default_model.features_in
cols_refusal = refusal_model.features_in
cols_combined = list(set(cols_default + cols_refusal))

with open('cols_combined.txt', 'w') as f:
    for c in cols_combined:
        f.write(c)
        f.write('\n')

In [23]:
accepted_filtered_df = default_model.filter_df(accepted_df)

Rows with 'account_age_days' column not positive will be discarded.


In [115]:
accepted_filtered_df['refusal'] = 0
refused_df['refusal'] = 1

df = pd.concat([accepted_filtered_df, refused_df])

In [25]:
df['status'].value_counts()

status
Refused         28579
Fully Paid       5901
Defaulted        3530
Restructured       76
Name: count, dtype: int64

In [120]:
df = df[cols_combined + ['status', 'dob', 'refusalReason']]

In [153]:
def sample_one(status=None):

    if status is not None:
        df_filtered = df[df['status']==status]
    else:
        df_filtered = df

    #df.drop(columns=['Id'], inplace=True)

    #sample = df.sample(1)

    sample_df = df_filtered.sample(1)

    sample = sample_df.reset_index().to_dict(orient='records')[0]

    if pd.isna(sample['refusalReason']):
        sample['refusalReason'] = ''

    sample['dob'] = sample['dob'][0:10]
    sample['request_date'] = '2024-04-15'

    return sample

In [154]:
def send_request(payload):
    headers = {
    "Content-type": "application/json",
    "Accept": "application/json"
    }

    response = requests.post(
        "https://westgate.fly.dev/predict",
        headers=headers,
        json=payload
    )

    return response.json()

In [162]:
payload

{'index': 19455,
 'sum_employment_insurance_income_previous_month': 0.0,
 'balance_min': -154.58,
 'count_distinct_micro_lenders': 0.0,
 'sum_micro_loan_payments_previous_month': 0.0,
 'count_days_negative_balance_90_days': 10.0,
 'count_loan_payments_previous_month': 0.0,
 'sum_employer_income_current_month': 0.0,
 'sum_total_income_current_month': 0.0,
 'average_monthly_micro_loan_payments_complex': 0.0,
 'recurring_deposits_2_months_ago': 0.0,
 'count_loan_payments_current_month': 0.0,
 'average_monthly_utility_payments_complex': 233.33,
 'employer_income_frequency': 'Unknown',
 'sum_employer_income_2_months_ago': 0.0,
 'sum_non_employer_income_2_months_ago': 1125.19,
 'sum_government_income_previous_month': 75.53,
 'sum_loan_deposits_30_days': 0.0,
 'sum_non_employer_income_current_month': 0.0,
 'sum_disability_income_current_month': 0.0,
 'sum_loan_payments_2_months_ago': 0.0,
 'average_closing_balance_day_after_employer_income': 622.31,
 'balance_current': 194.41,
 'sum_total_inc

In [160]:
#payload = sample_one('Fully Paid')
payload = sample_one('Refused')

In [161]:
send_request(payload)

{'refusal': {'score': 0.9521138072013855, 'percentile': 75},
 'default': {'score': 0.526416540145874, 'percentile': 95},
 'uw_decision': 'refuse',
 'version': 'v1.0',
 'timestamp': '15/04/2024 17:59:22'}